<a href="https://colab.research.google.com/github/leochame/Computer-Study-Note/blob/main/llm/llms-from-srcatch/ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
注意力机制是LLM的核心，那么接下来我们将组装剩下的组件

In [1]:
from importlib.metadata import version

import matplotlib
import tiktoken
import torch

print("matplotlib version:", version("matplotlib"))
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
#加载并确认版本

matplotlib version: 3.10.0
torch version: 2.6.0+cu124
tiktoken version: 0.9.0


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}
#初始化定义需要的各种超参数

上面是配置，

In [3]:
import torch
import torch.nn as nn


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # 词嵌入层，将输入索引转换为词向量，词表大小由字典大小和特征维度决定。
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # 位置信息嵌入层，基于文本长度和特征维度生成位置信息。
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        # Dropout 层，用于随机丢弃一部分嵌入信息以减少过拟合。

        # 使用多个 Transformer 块（占位符）
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        # Transformer 模块的堆叠，模型核心部分。

        # 使用归一化层（占位符）
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        # 最终归一化层，用于调整特征分布。

        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
        # 输出层，将特征映射到词表分布，最终预测输出单词。

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        # 获取批次大小和序列长度。

        tok_embeds = self.tok_emb(in_idx)
        # 根据输入索引生成词嵌入。
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        # 生成对应的位置信息嵌入。

        x = tok_embeds + pos_embeds
        # 将词嵌入和位置信息嵌入相加。
        x = self.drop_emb(x)
        # 应用 Dropout 随机丢弃部分信息。
        x = self.trf_blocks(x)
        # 通过多个 Transformer 块处理特征。
        x = self.final_norm(x)
        # 应用最终的归一化层。
        logits = self.out_head(x)
        # 将隐藏状态映射到词表分布，生成预测结果。
        return logits


class DummyTransformerBlock(nn.Module):
    # Transformer 块的占位类。
    def __init__(self, cfg):
        super().__init__()
        # 占位，实际模型应实现注意力机制和前馈网络。

    def forward(self, x):
        # 此块不执行任何操作，仅返回输入。
        return x


class DummyLayerNorm(nn.Module):
    # 归一化层的占位类。
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 参数用于模拟 LayerNorm 的接口。

    def forward(self, x):
        # 此层不执行任何操作，仅返回输入。
        return x